In [1]:
from sys import path
path.insert(0, '/home/azadoks/git/glosim2/')
path.insert(0, '/home/azadoks/git/structureREST/lib/')
from collections import OrderedDict
from pymatgen import MPRester
from libmatch.soap import get_Soaps
import numpy as np
import zoap
# import distances

np.set_printoptions(floatmode='fixed', precision=4)

/home/azadoks/miniconda3/envs/python2/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


In [ ]:
material_ids = OrderedDict([
    ('diamond', 'mp-66'),  # C
    ('gaas', 'mp-2534'),  # GaAs
    ('rocksalt', 'mp-22862'),  # NaCl
    ('cubic_perovskite', 'mp-2998'),  # BaTiO3
    ('tetragonal_perovskite', 'mp-5986'),  # BaTiO3
    ('trigonal_perovskite', 'mp-5020'),  # BaTiO3
    ('orthorhombic_perovskite', 'mp-5777'),  # BaTiO3
    ('zincblende', 'mp-10695'),  # ZnS
    ('wurtzite', 'mp-10281'),  # ZnS
    ('fcc', 'mp-23'),  # Ni
    ('big_fcc', 'mp-76'),  # Sr
    ('bcc', 'mp-13'),  # Fe
    ('big_bcc', 'mp-70'),  # Rb
    ('hcp', 'mp-153'),  # Mg
    ('trigonal', 'mp-782'),  # Te2Pd
    ('tetragonal', 'mp-742'),  # Ti2Cu
    ('monoclinic', 'mp-684'),  # BaS2
    ('triclinic', 'mp-9122'),  # CaP3
    ('orthorhombic', 'mp-872')  # BaSn
])

In [2]:
material_ids = OrderedDict([
    ('zincblende', 'mp-10695'),  # ZnS
#     ('wurtzite', 'mp-10281'),  # ZnS
    ('fcc', 'mp-23'),  # Ni
    ('big_fcc', 'mp-76'),  # Sr
    ('bcc', 'mp-13'),  # Fe
])

In [7]:
material_ids = OrderedDict([
    ('zincblende', 'mp-10695'),  # ZnS
    ('wurtzite', 'mp-10281'),  # ZnS
])

In [ ]:
# py3
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures = OrderedDict([(name, mpr.get_structure_by_material_id(id))
                              for name, id in material_ids.items()])

In [8]:
# py2
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures = OrderedDict([(name, mpr.get_structure_by_material_id(id))
                              for name, id in material_ids.iteritems()])

## pymatgen
---

In [ ]:
pmg_distances = np.zeros((len(structures), len(structures)))
for i, a in enumerate(structures.values()):
    for j, b in enumerate(structures.values()):
        d = distances.pymatgen_distances(structures=[a.as_dict(), b.as_dict()])[0]
        if d is None:
            pmg_distances[i, j] = d
        else:
            pmg_distances[i, j] = d[0]

In [ ]:
display(material_ids.keys())
display(pmg_distances)

## matminer
---

In [ ]:
mat_distances = np.zeros((len(structures), len(structures)))
for i, a in enumerate(structures.values()):
    for j, b in enumerate(structures.values()):
        d = distances.matminer_distances(structures=[a.as_dict(), b.as_dict()])[0]
        mat_distances[i, j] = d

In [ ]:
display(material_ids.keys())
display(mat_distances)

## SOAP
---

In [9]:
qps = OrderedDict([(name, zoap.structure2quippy(structure,
                                                anonymize=True, scale=True,
                                                standardize=False, primitivize=False)
                   )
                   for name, structure in structures.iteritems()])

soaps = get_Soaps(qps.values(),
                  nocenters=None, chem_channels=False,
                  centerweight=1.0, gaussian_width=0.25,
                  cutoff=2.5, cutoff_transition_width=0.25,
                  nmax=12, lmax=8,
                  spkitMax=None, nprocess=8,
                  chemicalProjection=None, dispbar=False,
                  is_fast_average=False)

In [10]:
soap_distances = np.zeros((len(soaps), len(soaps)))
for i, a in enumerate(soaps):
    a = np.mean(a.values(), axis=0)
    for j, b in enumerate(soaps):
        b = np.mean(b.values(), axis=0)
        d = zoap.average_distance(a, b)
        if np.isnan(d):
            soap_distances[i, j] = 0
        else:
            soap_distances[i, j] = d

/home/azadoks/git/structureREST/lib/zoap.py:89: RuntimeWarning: invalid value encountered in sqrt
  d12 = np.sqrt(2 - 2 * (k12 / np.sqrt(k11 * k22)))


In [11]:
display(material_ids.keys())
display(soap_distances)

['zincblende', 'wurtzite']

array([[0.0000, 0.4711],
       [0.4711, 0.0000]])